<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#使用生成器为-Keras-训练提供数据" data-toc-modified-id="使用生成器为-Keras-训练提供数据-1">使用生成器为 Keras 训练提供数据</a></span><ul class="toc-item"><li><span><a href="#1.-导入数据" data-toc-modified-id="1.-导入数据-1.1">1. 导入数据</a></span></li><li><span><a href="#2.-数据预处理" data-toc-modified-id="2.-数据预处理-1.2">2. 数据预处理</a></span></li><li><span><a href="#3.-定义模型" data-toc-modified-id="3.-定义模型-1.3">3. 定义模型</a></span></li><li><span><a href="#4.-开始训练" data-toc-modified-id="4.-开始训练-1.4">4. 开始训练</a></span></li><li><span><a href="#5.-训练结束后评估" data-toc-modified-id="5.-训练结束后评估-1.5">5. 训练结束后评估</a></span></li></ul></li></ul></div>

## 使用生成器为 Keras 训练提供数据

这一部分我们学习使用生成器为 Keras 提供训练和验证数据.

持续训练模型可能有过拟合的风险, 一种解决方法是提前停止训练, 因此我们可以在训练的过程中加入验证, 当验证精度长时间不再提升的时候就可以停止训练了.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K

print(tf.VERSION, K.__version__)
nn = K.layers

1.13.1 2.2.4-tf


### 1. 导入数据

In [2]:
fashion_mnist = K.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


### 2. 数据预处理

我们定义一个数据预处理函数. 这里为了简单只做灰度值归一化的操作.

In [3]:
def preprocess(image, label):
    image = (image / 255.).astype(np.float32)
    label = label.astype(np.int32)
    return image, label

接下来我们想使用 Keras 的 `fit_generator` 函数来开启训练, 因此要先定义一个数据生成器. 这个数据生成器接受 images 数组和 labels 数组作为输入, 同时可以指定批大小 `batch_size`, 和是否要打乱数据 `shuffle=True/False` 

In [4]:
def data_loader(images, labels, batch_size=4, shuffle=True):
    assert images.shape[0] == labels.shape[0], \
        "Shape mismatch: images {} vs labels {}".format(images.shape, labels.shape)
    images, labels = preprocess(images, labels)

    while True:
        all_indices = np.arange(images.shape[0])
        if shuffle:
            np.random.shuffle(all_indices)
        for i in range(0, all_indices.shape[0], batch_size):
            image_batch = images[all_indices[i:i + batch_size]]
            label_batch = labels[all_indices[i:i + batch_size]]
            yield image_batch, label_batch

### 3. 定义模型

这里我们仍然定义包含一个隐藏层的多层感知机(MLP)

In [5]:
model = K.Sequential([
    nn.Flatten(input_shape=(28, 28)),
    nn.Dense(128, activation=tf.nn.relu),
    nn.Dense(10, activation=tf.nn.softmax)
])

Instructions for updating:
Colocations handled automatically by placer.


定义模型优化器, 损失函数和评估指标

In [6]:
model.compile(tf.train.AdamOptimizer(),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

### 4. 开始训练

首先创建用于训练和验证的数据生成器

In [16]:
batch_size = 16
train_gen = data_loader(train_images, train_labels, batch_size=batch_size)
val_gen = data_loader(test_images, test_labels, batch_size=batch_size, shuffle=False)

然后我们使用 `fit_generator` 函数, 同时提供训练生成器和验证生成器. Keras 会在每个 epoch 结束的时候评估验证集的数据, 并输出验证集上的准确率.

In [8]:
model.fit_generator(train_gen, steps_per_epoch=train_labels.shape[0] // batch_size, epochs=10,
                    validation_data=val_gen, validation_steps=test_labels.shape[0] // batch_size)

Epoch 1/10
3750/3750 [==============================] - 14s 4ms/step - loss: 0.4812 - acc: 0.8270 - val_loss: 0.4007 - val_acc: 0.8558
Epoch 2/10
3750/3750 [==============================] - 13s 4ms/step - loss: 0.3712 - acc: 0.8642 - val_loss: 0.3844 - val_acc: 0.8610
Epoch 3/10
3750/3750 [==============================] - 14s 4ms/step - loss: 0.3315 - acc: 0.8787 - val_loss: 0.3871 - val_acc: 0.8552
Epoch 4/10
3750/3750 [==============================] - 16s 4ms/step - loss: 0.3093 - acc: 0.8860 - val_loss: 0.3643 - val_acc: 0.8730
Epoch 5/10
3750/3750 [==============================] - 17s 4ms/step - loss: 0.2911 - acc: 0.8923 - val_loss: 0.3492 - val_acc: 0.8774
Epoch 6/10
3750/3750 [==============================] - 18s 5ms/step - loss: 0.2778 - acc: 0.8959 - val_loss: 0.3335 - val_acc: 0.8799
Epoch 7/10
3750/3750 [==============================] - 16s 4ms/step - loss: 0.2653 - acc: 0.9012 - val_loss: 0.3485 - val_acc: 0.8807
Epoch 8/10
3750/3750 [==============================] -

### 5. 训练结束后评估

这里我们评估也使用生成器

In [17]:
test_loss, test_acc = model.evaluate_generator(val_gen, steps=test_labels.shape[0] // batch_size)
print('Test accuracy:', test_acc)

Test accuracy: 0.8799


使用数组输入评估, 要记得做同样的数据预处理.

In [14]:
test_images_, test_labels_ = preprocess(test_images, test_labels)
test_loss, test_acc = model.evaluate(test_images_, test_labels_)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 41us/sample - loss: 0.3483 - acc: 0.8799
Test accuracy: 0.8799


结果肯定是一样的.